In [ ]:
library(tidyverse)
library(fs)
library(extrafont)
library(ggtext)

source('./../../r-utils/plots.r')

## Transitions

In [ ]:
saturation_df =
    dir_ls('exports/', regexp = '^exports/router-saturation-.*\\.csv$') %>%
    set_names(., str_extract(path_file(.), '(?<=^router-saturation-).*(?=\\.csv$)')) %>%
    imap(function(x, i) {
        read_csv(x) %>% mutate(model = i)
    }) %>%
    list_rbind()

head(saturation_df)

In [ ]:
plot = 
    saturation_df %>%
    filter(pct_dims <= 40) %>%
    mutate(accuracy = accuracy * 100) %>%
    mutate(model = fct_relevel(model, 'olmoe', 'qwen1.5moe', 'dsv2', 'qwen3moe')) %>%
    ggplot() + 
    geom_line(aes(x = pct_dims, y = accuracy, color = model, group = model), linewidth = 1.0) +
    geom_point(aes(x = pct_dims, y = accuracy, color = model, group = model), size = 2.0) +
    scale_color_manual(
        labels = c(`olmoe` = 'OlMoE', `qwen1.5moe` = 'Qwen1.5MoE', `dsv2` = 'DSv2-Lite', `qwen3moe` = 'Qwen3MoE'),
        values = c(`olmoe` = '#544fc5', `qwen1.5moe` = '#00e272', `dsv2` = '#fe6a35', `qwen3moe` = '#2caffe')
    ) +
    scale_x_continuous(
        breaks = seq(0, 100, 8),
        labels = function(x) paste0(x, '%')
    ) +
    # scale_x_log10(
    #     limits = c(0.001, 20),  # Focus on 0.5% to 20%
    #     breaks = c(0, 0.1, 1, 2, 5, 10, 20),
    #     labels = function(x) paste0(x, '%')
    # ) +
    scale_y_continuous(
        limits = c(0, 100),
        breaks = seq(0, 100, 20),
        labels = function(x) paste0(x, '%')
    ) +
    labs(title = NULL, color = NULL, x = 'Fraction of highest-magnitude dimensions', y = 'Probe accuracy') +
    theme_iclr(base_size = 11)

ggsave(
    str_glue('exports/router-saturation-md.pdf'),
    plot = plot, 
    width = 7, height = 3.0,
    units = 'in', dpi = 300,
    device = cairo_pdf
)
ggsave(
    str_glue('exports/router-saturation-md.png'),
    plot = plot, 
    width = 7, height = 3.0,
    units = "in", dpi = 300
)

plot